In [1]:
import re
from latexcompiler import LC
import os
from pathlib import Path
import json
import pandas as pd
def load_json(path):
    with open(path,"r") as file:
        dict=json.load(file)
    return dict

def load_text(path):
    with open(path,"r") as file:
        template="".join(file.readlines())
    return template

class Invoice_filler():
    tags={
        "tax_rate":"#TAX_RATE",
        "invoice_ref":"#INVOICE_REF",
        "due_time":"#DUE_TIME",
        "project_name":"#PROJECT_NAME",
        "logo_path":"#LOGO_PATH",
        "company_info":"#COMPANY_INFO",
        "client_info":"#CLIENT_INFO",
        "terms_conditions":"#TERMS_CONDITIONS",
        "invoice_items":"#INVOICE_ITEMS"
    }
    def __init__(self,
                 template_snippets_path:str,
                 template_path:str,
                 company_info:dict,
                 client_info:dict,
                 tax_info:dict,
                 logo_path:str,
                 terms_conditions:str) -> None:
        self.tempalte=load_text(template_path)
        self.company_info=company_info
        self.client_info=client_info
        self.tax_info=tax_info
        self.logo_path=logo_path
        self.terms_conditions=terms_conditions
        self.template_snippets=load_json(template_snippets_path)

    def _render_invoice_items(self,invoice_data):
        """
            Creates partial tex code for the given invoice data.
        """
        res=""
        for task in invoice_data:
            t=task
            for item in invoice_data[task]:
                snippet=r"\\"+self.template_snippets["items"]["item"]
                print(item)
                snippet=self._fill_snippet("#1",t,snippet)
                snippet=self._fill_snippet("#2",item["detail"],snippet)
                snippet=self._fill_snippet("#3",item["qt"],snippet)
                snippet=self._fill_snippet("#4",item["unit"],snippet)
                snippet=self._fill_snippet("#5",item["total"],snippet)
                t=r""
                res+=snippet+"\n"
            res+=r"\\"+self.template_snippets["items"]["separator"]+"\n"
        print(res)
        return res

    def _render_business_info(self,business_info:dict,actor_type:str)->str:
        """
        """
        res=""
        for key in business_info:
            snippet=r"\\"+self.template_snippets[actor_type][key]
            snippet=self._fill_snippet("#",business_info[key],snippet)
            res+=snippet
        print(res)
        return res

    def _fill_snippet(self,search,data,snippet,prefix=""):
        a=re.sub(search,f"{prefix}{data}",snippet)
        return a

    def _fill_template(self,ref:str,due_time:int,project_name:str,invoice_data:dict)->str:
        """
        """
        res=self.tempalte
        # res=self._fill_snippet(self.tags["tax_rate"],self.tax_rate,res)
        res=self._fill_snippet(self.tags["invoice_ref"],ref,res)
        res=self._fill_snippet(self.tags["due_time"],str(due_time),res)
        res=self._fill_snippet(self.tags["project_name"],project_name,res)
        res=self._fill_snippet(self.tags["logo_path"],self.logo_path,res)
        res=self._fill_snippet(self.tags["company_info"],self._render_business_info(self.company_info,"company"),res)
        res=self._fill_snippet(self.tags["client_info"],self._render_business_info(self.client_info,"client"),res)
        res=self._fill_snippet(self.tags["terms_conditions"],self.terms_conditions,res)
        res=self._fill_snippet(self.tags["invoice_items"],self._render_invoice_items(invoice_data),res)
        return res

    def _save_latex(self,render_path,filled_template):
        doc_path=render_path+"/temp.tex"
        with open(doc_path,"w") as file:
            file.write(filled_template)
        return doc_path

    def compile_document(self,render_path):
        path=os.path.abspath(render_path)

        LC.compile_document(tex_engine="pdflatex",
                            bib_engine='biber',
                            no_bib=True,
                            path=path,
                            folder_name=Path(path).parent)
    
    def render_document(self,ref:str,due_time:int,project_name:str,invoice_data:dict,render_dir:str):
        filled_template=self._fill_template(ref,due_time,project_name,invoice_data)
        doc_path=self._save_latex(render_dir,filled_template)
        self.compile_document(doc_path)


In [2]:
            
            

template_snippets_path=r"C:\Users\pc\Documents\programming\personal_projects\invoice_creator\templates\snippets.json"
template_path=r"C:\Users\pc\Documents\programming\personal_projects\invoice_creator\templates\main.tex"
logo_path=r"C:/Users/pc/Documents/programming/personal_projects/invoice_creator/templates/logoSmaller.png"

business_info={'NAME': 'PRISgazouz1', 
               'ADRESS': 'testadrs', 
               'TEL': '012384712', 
               'RC': 'r10238947', 
               'NIF': 'n239148239', 
               'AI': 'a2391847239', 
               'RIB': '123148976234897'}
clinet_info={'NAME': 'PRISMA1', 
             'ADRESS': 'aaaa', 
             'TEL': 12384712, 
             'RC': 'r10238947', 
             'NIF': 'n2391482391', 
             'AI': 'a2391847239', 
             'RIB': 123148976234897}
tax_info={'test': 4140.0}

filler=Invoice_filler(
    template_snippets_path=template_snippets_path,
    template_path=template_path,
    company_info=business_info,
    client_info=clinet_info,
    tax_info=tax_info,
    logo_path=logo_path,
    terms_conditions="nothing"
)

In [ ]:
invoice_info={'ref': '9/24', 'date': '08/10/2024', 'due_date': '08/10/2024', 'project_name': 'stest'}
items_info={'Animation': [{'detail': 'four', 'unit': 7000, 'qt': 1, 'total': 7000.0}], 
            'Modeling beach': [{'detail': 'three', 'unit': 7000, 'qt': 1, 'total': 7000.0}], 
            'Modeling buildings': [{'detail': 'one', 'unit': 10000, 'qt': 2, 'total': 20000.0}], 
            'Modeling environment': [{'detail': 'two ', 'unit': 5000, 'qt': 1, 'total': 5000.0}], 
            'Track': [{'detail': 'test', 'unit': 15000, 'qt': 1, 'total': 15000.0}, 
                      {'detail': 'second_task', 'unit': 15000, 'qt': 1, 'total': 15000.0}]
            }


filler.render_document(invoice_info["ref"],
                        due_time=90,
                        project_name=invoice_info["project_name"],
                        invoice_data=items_info,
                        render_dir=r"C:\Users\pc\Documents\programming\personal_projects\invoice_creator\templates/"
                        )


\\Entreprise{PRISgazouz1}\\EAdresse{testadrs}\\ETel{012384712}\\ERC{r10238947}\\ENIF{n239148239}\\EAI{a2391847239}\\ERIB{123148976234897}
\\Destinataire{PRISMA1}\\DAdresse{aaaa}\\DTel{12384712}\\DRC{r10238947}\\DNIF{n2391482391}\\DAI{a2391847239}\\DRIB{123148976234897}
{'detail': 'four', 'unit': 7000, 'qt': 1, 'total': 7000.0}
{'detail': 'three', 'unit': 7000, 'qt': 1, 'total': 7000.0}
{'detail': 'one', 'unit': 10000, 'qt': 2, 'total': 20000.0}
{'detail': 'two ', 'unit': 5000, 'qt': 1, 'total': 5000.0}
{'detail': 'test', 'unit': 15000, 'qt': 1, 'total': 15000.0}
{'detail': 'second_task', 'unit': 15000, 'qt': 1, 'total': 15000.0}
\\invoiceitem{Animation}{four}{1}{7000}{7000.0}
\\separator
\\invoiceitem{Modeling beach}{three}{1}{7000}{7000.0}
\\separator
\\invoiceitem{Modeling buildings}{one}{2}{10000}{20000.0}
\\separator
\\invoiceitem{Modeling environment}{two }{1}{5000}{5000.0}
\\separator
\\invoiceitem{Track}{test}{1}{15000}{15000.0}
\\invoiceitem{}{second_task}{1}{15000}{15000.0}
\\

Compile LaTeX file(s):   0%|          | 0/2 [00:00<?, ?it/s]

In [142]:
df1=df.copy()
df1["total"]=df1["unit"]*df1["qt"]
df

,task,detail,unit,qt
0,Track,test,15000,1
1,Track,second_task,15000,1
2,Modeling buildings,one,10000,2
3,Modeling environment,two,5000,1
4,Modeling beach,three,7000,1
5,Animation,four,7000,1


In [133]:
res={key:{}  for key in df.groupby("task").groups}

for key in df.groupby("task").groups:
    items=df.groupby("task").get_group(key)
    for _,task in items.iterrows():
        res[key]=task[task!="task"].drop("task").to_dict()

print(res)

{'Animation': {'detail': 'four', 'unit': 7000, 'qt': 1}, 'Modeling beach': {'detail': 'three', 'unit': 7000, 'qt': 1}, 'Modeling buildings': {'detail': 'one', 'unit': 10000, 'qt': 2}, 'Modeling environment': {'detail': 'two ', 'unit': 5000, 'qt': 1}, 'Track': {'detail': 'second_task', 'unit': 15000, 'qt': 1}}


In [123]:
for _,task in df.groupby("task").get_group("Track").iterrows():
    print(task[task!="task"].drop("task").to_dict())


{'detail': 'test', 'unit': 15000, 'qt': 1}
{'detail': 'second_task', 'unit': 15000, 'qt': 1}


detail    second_task
unit            15000
qt                  1
Name: 1, dtype: object

In [6]:

company_info=pd.read_csv(r"./company_info.csv")

In [52]:
assert company_info.shape==(7,2)
{row.iloc[0]:row.iloc[1]  for a,row in company_info.iterrows()}

{'NAME': 'PRISgazouz1',
 'ADRESS': 'testadrs',
 'TEL': '012384712',
 'RC': 'r10238947',
 'NIF': 'n239148239',
 'AI': 'a2391847239',
 'RIB': '123148976234897'}

In [27]:
os.path.splitext("./test.json")[-1]

'.json'

In [26]:
os.path.splitext(os.path.basename("./test/one.txt"))

('one', '.txt')

In [85]:
v=[
128000,
-1000,
-1000,
-1000,
10000
]
sum(v)

135000